In [1]:
import syft as sy
from syft.store.blob_storage import BlobStorageConfig, BlobStorageClientConfig
from syft.store.blob_storage.seaweedfs import SeaweedFSClient, SeaweedFSClientConfig
from syft import ActionObject
from syft.service.action.action_data_empty import ActionFileData
from syft.service.queue.zmq_queue import ZMQQueueConfig, ZMQClientConfig
from collections import defaultdict
# from syft import BlobFile

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /Users/koen/workspace/pysyft


In [2]:
sy.stage_protocol_changes()

Exception: Can't add {'version': 2, 'hash': '29a82afcb006a044b6ae04c6ea8a067d145d28b4210bb038ea9fa86ebde108c8', 'action': 'add'} already in state {'2': {'version': 2, 'hash': '29a82afcb006a044b6ae04c6ea8a067d145d28b4210bb038ea9fa86ebde108c8', 'action': 'add'}}

In [ ]:
%debug

> /Users/koen/workspace/PySyft/packages/syft/src/syft/protocol/data_protocol.py(117)build_state()
    115                         or hash_str in state_version_hashes
    116                     ):
--> 117                         raise Exception(
    118                             f"Can't add {object_metadata} already in state {versions}"
    119                         )

ipdb> stop_key
ipdb> self.protocol_history
{'1': {'object_versions': {'PartialSyftObject': {'1': {'version': 1, 'hash': '008917584d8e1c09015cdbef02f59c0622f48e0618877c1b44425c8846befc13', 'action': 'add'}}, 'NodeMetadataUpdate': {'1': {'version': 1, 'hash': '569d124c23590360bda240c19b53314ccc6204c5d1ab0d2898976a028e002191', 'action': 'add'}}, 'NodeMetadata': {'1': {'version': 1, 'hash': '6bee018894dfdf697ea624740d0bf051750e0b0d8470ced59646f6d8812068ac', 'action': 'add'}, '2': {'version': 2, 'hash': 'f856169fea72486cd436875ce4411ef935da11eb7c5af48121adfa00d4c0cdb6', 'action': 'add'}, '3': {'version': 3, 'hash': '3cc67

ipdb> self.protocol_history.keys()
dict_keys(['1', 'dev'])


In [2]:
node = sy.orchestra.launch(name="test-domain-helm2", dev_mode=True,
                           reset=True,
                           n_consumers=4,
                           create_producer=True)
client = node.login(email="info@openmined.org", password="changethis")

Staging Protocol Changes...
Logged into <test-domain-helm2: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

```bash
docker run --entrypoint /bin/sh -p 8333:8333 -p 8888:8888 chrislusf/seaweedfs -c "echo 's3.configure -access_key admin -secret_key admin -user iam -actions Read,Write,List,Tagging,Admin -apply' | weed shell > /dev/null 2>&1 & weed server -s3 -s3.port=8333 -master.volumeSizeLimitMB=2048"
```

In [3]:
blob_config = BlobStorageConfig(client_type=SeaweedFSClient,
                                client_config=SeaweedFSClientConfig(host="http://0.0.0.0",
                                                                    port="8333",
                                                                    access_key="admin",
                                                                    secret_key="admin",
                                                                    bucket_name="test_bucket",
                                                                    region="us-east-1"
                                                                   )
)

In [4]:
node.python_node.init_blob_storage(blob_config)

# Inputs

In [5]:
# TODO: fix way we send list of files
scenario_objs = ActionObject.from_obj([
    BlobFile.upload_from_path("scenario_data.jsonl", client)
])

scenario_files_ptr = scenario_objs.send(client)

In [8]:
# scenario_objs[0]

In [9]:
input_files = ActionObject.from_obj([
    BlobFile.upload_from_path("short_input.jsonl", client)
])

In [10]:
input_files_dataset = sy.Dataset(
    name="Helm dataset",
    asset_list=[
        sy.Asset(
            name="helm train data",
            data=input_files,
            mock=sy.ActionObject.empty()
        )
    ]
)

In [11]:
client.upload_dataset(input_files_dataset)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  7.43it/s]

Uploading: helm train data


SyftSuccess: Dataset uploaded to 'test-domain-helm2'. To see the datasets uploaded by a client on this node, use command `[your_client].datasets`

In [12]:
input_files_asset = client.datasets["Helm dataset"].assets[0]

# Syft functions

In [13]:
@sy.syft_function()
def compute_document_data_overlap(domain, scenario_file, input_files, n):
    print("starting overlap computation")
    from nltk import ngrams
    from collections import defaultdict
    from string import punctuation
    import re, json

    r = re.compile(r"[\s{}]+".format(re.escape(punctuation)))
    
    def create_ngram_index(light_scenarios, n_values, stats_key_counts):
        ngram_index = {n:{}  for n in n_values}
        for scenario in light_scenarios:
            for n in n_values:
                stats_key = scenario['scenario_key'] + '_' + str(n)
                stats_key_counts[stats_key] = len(scenario['instances'])
                for instance in scenario['instances']:
                    id = instance['id']                    
                    input_tokens = r.split(instance['input'].lower())
                    for input_ngram in ngrams(input_tokens, n):
                        if input_ngram not in ngram_index[n]:
                            ngram_index[n][input_ngram] = set()
                        ngram_index[n][input_ngram].add(stats_key + '+' + id + '+' + 'input')

                    # compute reference ngrams
                    for reference in instance['references']:
                        reference_unigrams = r.split(reference.lower())
                        for reference_ngram in ngrams(reference_unigrams, n):
                            if reference_ngram not in ngram_index[n]:
                                ngram_index[n][reference_ngram] = set()
                            ngram_index[n][reference_ngram].add(stats_key + '+' + id + '+' + 'references')
        return ngram_index
    
    # # SETUP
    print("preparing scenarios and creating indexes")
    light_scenarios = []
    for light_scenario_json in scenario_file.iter_lines():
        light_scenario_dict: dict = json.loads(light_scenario_json)

        light_scenario_key_dict: dict = light_scenario_dict["scenario_key"]
        subject_spec = light_scenario_key_dict["scenario_spec"]['args']['subject']
        light_scenario_key = subject_spec + '_' + light_scenario_key_dict["split"]
        light_instances = [
            {
                'input': instance_dict['input'], 
                'references': instance_dict['references'], 
                'id': instance_dict["id"]
            }
            for instance_dict in light_scenario_dict["instances"]
        ]
        light_scenarios.append({'scenario_key': light_scenario_key, 'instances': light_instances})
        
    stats_key_counts = defaultdict(int)
    
    ngram_index = create_ngram_index(
        light_scenarios=light_scenarios, n_values=[n], stats_key_counts=stats_key_counts
    )
    
    r = re.compile(r"[\s{}]+".format(re.escape(punctuation)))
    stats_key_to_input_ids = defaultdict(set)
    stats_key_to_reference_ids = defaultdict(set)
    print("computing overlap")
    from time import sleep
    sleep(1)
    
    domain.init_progress(input_files[0].file_size)

    for input_file in input_files:
        for bytes_read, line in input_file.iter_lines(progress=True):
            sleep(1)
            document = json.loads(line)["text"]
            document_tokens = r.split(document.lower())
            for n in ngram_index.keys():
                for document_ngram in ngrams(document_tokens, n):
                    if document_ngram in ngram_index[n]:
                        for entry_overlap_key in ngram_index[n][document_ngram]:
                            stats_key, id, part = entry_overlap_key.split("+")
                            if part == "input":
                                stats_key_to_input_ids[stats_key].add(id)
                            elif part == "references":
                                stats_key_to_reference_ids[stats_key].add(id)
            domain.set_progress(bytes_read)
    print("done")
    
    return stats_key_to_input_ids, stats_key_to_reference_ids, stats_key_counts

SyftSuccess: Syft function 'compute_document_data_overlap' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [14]:
client.code.submit(compute_document_data_overlap)

SyftSuccess: User Code Submitted

In [15]:
@sy.syft_function_single_use(input_files=input_files_asset, scenario_files=scenario_files_ptr)
def main_function(domain, input_files, scenario_files):
    N = [5, 9, 13]
    jobs = []
    for n in N[:1]:
        for scenario_file in scenario_files:
            batch_job = domain.launch_job(
                compute_document_data_overlap,
                scenario_file=scenario_file,
                input_files=input_files,
                n=n
            )
            jobs.append(batch_job)

    return None


SyftSuccess: Syft function 'main_function' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [16]:
client.code.request_code_execution(main_function)
client.requests[-1].approve()

Request approved for domain test-domain-helm2


SyftSuccess: Request a929166615674eeea593cd3100e4d9d0 changes applied

In [17]:
job = client.code.main_function(input_files=input_files_asset,
                                scenario_files=scenario_files_ptr,
                                blocking=False)

# Get results

In [18]:
job

```python
class Job:
    id: UID = 9bb9df1984c8415e9567b755f60ea0d2
    status: created
    has_parent: False
    result: None
    logs:

0 
    
```

In [21]:
job.subjobs

[syft.service.job.job_stash.Job]

In [22]:
client.jobs

14/11/23 11:52:44 FUNCTION LOG (927bafe374de4d35a13384c9ed39726a): preparing scenarios and creating indexes
14/11/23 11:52:44 FUNCTION LOG (927bafe374de4d35a13384c9ed39726a): computing overlap


In [23]:
job.subjobs[0].logs()

starting overlap computation
preparing scenarios and creating indexes
computing overlap



14/11/23 11:52:55 FUNCTION LOG (927bafe374de4d35a13384c9ed39726a): done


In [24]:
results = [j.wait().get() for j in job.subjobs]

In [25]:
#stats_key_to_input_ids, stats_key_to_reference_ids, stats_key_counts
results

[(defaultdict(set,
              {'philosophy_test_5': {'id328'},
               'philosophy_valid_5': {'id12'}}),
  defaultdict(set, {}),
  defaultdict(int,
              {'philosophy_train_5': 5,
               'philosophy_valid_5': 34,
               'philosophy_test_5': 311,
               'anatomy_train_5': 5,
               'anatomy_valid_5': 14,
               'anatomy_test_5': 135}))]

In [26]:
results[0]

(defaultdict(set,
             {'philosophy_test_5': {'id328'}, 'philosophy_valid_5': {'id12'}}),
 defaultdict(set, {}),
 defaultdict(int,
             {'philosophy_train_5': 5,
              'philosophy_valid_5': 34,
              'philosophy_test_5': 311,
              'anatomy_train_5': 5,
              'anatomy_valid_5': 14,
              'anatomy_test_5': 135}))

# Aggregate

In [24]:
stats_key_to_input_ids, stats_key_to_reference_ids, stats_key_counts = zip(*results)

total_input_ids = defaultdict(set)
total_reference_ids = defaultdict(set)
total_stats_key_counts = defaultdict(int)

for d in stats_key_counts:
    for key, val in d.items():
        total_stats_key_counts[key] += val


for d in stats_key_to_input_ids:
    for key in d:
        new_set = set()
        if key in total_input_ids:
            new_set = total_input_ids[key]
        new_set = new_set.union(d[key])
        total_input_ids[key] = new_set

for d in stats_key_to_reference_ids:
    for key in d:
        new_set = set()
        if key in total_reference_ids:
            new_set = total_reference_ids[key]
        new_set = total_reference_ids[key].union(d[key])
        total_reference_ids[key] = new_set

all_data_overlap_stats = []
for stats_key, count in total_stats_key_counts.items():
    data_overlap_stats = {
        'data_overlap_stats_key': None,
        'num_instances': count,
        'instance_ids_with_overlapping_input': sorted(total_input_ids[stats_key]),
        'instance_ids_with_overlapping_reference': sorted(total_reference_ids[stats_key]),
    }
    subject, split, n_str = stats_key.split('_')
    data_overlap_stats['data_overlap_stats_key'] = {
        'light_scenario_key': {'subject': subject, 'split': split},
        'overlap_protocol_spec': {'n': int(n_str)}
    }
    all_data_overlap_stats.append(data_overlap_stats)




In [25]:
from pprint import pprint
pprint(all_data_overlap_stats)

[{'data_overlap_stats_key': {'light_scenario_key': {'split': 'train',
                                                    'subject': 'philosophy'},
                             'overlap_protocol_spec': {'n': 5}},
  'instance_ids_with_overlapping_input': [],
  'instance_ids_with_overlapping_reference': [],
  'num_instances': 5},
 {'data_overlap_stats_key': {'light_scenario_key': {'split': 'valid',
                                                    'subject': 'philosophy'},
                             'overlap_protocol_spec': {'n': 5}},
  'instance_ids_with_overlapping_input': ['id12'],
  'instance_ids_with_overlapping_reference': [],
  'num_instances': 34},
 {'data_overlap_stats_key': {'light_scenario_key': {'split': 'test',
                                                    'subject': 'philosophy'},
                             'overlap_protocol_spec': {'n': 5}},
  'instance_ids_with_overlapping_input': ['id328'],
  'instance_ids_with_overlapping_reference': [],
  'num_instances': 3